# Exercise 1: Groundwater Flow Model 

Elco Luijendijk, University of Bergen, Jan. 2026

**Intended audience:** Upper level BSc or MSc Earth Science


## Learning outcomes
By the end of this notebook you should be able to:
1. Construct your first own groundwater model in Python, using the finite-difference method.
2. Verify a numerical model against an analytical solution.
3. Interpret boundary conditions and their physical meaning.
4. Evaluate model sensitivity to grid size and parameter changes.

## How to work through this notebook
- Run cells from top to bottom.
- If results look odd, use Kernel → Restart & Run All.
- Save your notebook regularly.

## What to submit:
- Your completed Jupyter notebook (`.ipynb` file)
- A short word document with answers to all questions in the assignments

## Grading
- This exercise will count towards your final grade. All the exercises in the computer modelling part of GEOV212 will make up a total of 40% of the final grade.
  
## Tips for success:
- Save your notebook frequently
- Answer questions as you go (do not wait until the end)
- Make sure the notebook that you hand in runs and does not generate errors. You can check this by choosing restart and run-all, and then check for errors.
- Do not hesitate to ask for help if you do not understand something, or something does not work or there is another issue that you need help with
- If you have problems with meeting the deadline due to illness or other issues, please contact us as soon as possible so that we can accomodate this


Good luck!

## Part 1: Theory Overview

### Mathematical background

This exercise uses the **finite difference method** to solve the 1D steady-state groundwater flow equation. The detailed mathematical derivations are in a companion notebook:

**📓 [groundwater_theory_derivations.ipynb](groundwater_theory_derivations.ipynb)**

Work through the theory notebook if you want to understand:
- How Euler's method approximates derivatives
- Derivation of the groundwater flow equation
- How the finite difference approximation is derived
- The analytical solution for validation

### Key equation for this exercise

In this notebook we will use the following equation to simulate groundwater flow in a cross-section through the subsurface.

$$h(x) = \frac{1}{2} \left( \frac{Wb \Delta x^2}{K b} + h(x+\Delta x) + h(x - \Delta x) \right)$$

The equation calculates hydraulic head (h) at a position x by subdividing the subsurface in grid cells, and looking at the values of hydraulic head in the neighboring grid cells to the left and right, and the amount of groundwater recharge that is added to each grid cell.

The equation uses the following variables:

- x = distance from the left-hand side of the model
- $\Delta x$ = grid spacing (m)
- $b$ = aquifer thickness (m)
- $h$ = hydraulic head (m)
- $K$ = hydraulic conductivity (m/s)
- $W$ = source term (s⁻¹) representing groundwater recharge or pumping

Note: h(x) denotes h at position x

### Groundwater flow to a stream

Now let's apply this solution to a groundwater flow problem. Below is a conceptual model of groundwater flow in a permeable rock unit that is bounded by a stream on the left, and is fed by groundwater recharge (shown by $R$). The hydraulic head is a function of the amount of recharge that enters the system, the frictional resistance that the porous rocks exert on the groundwater and the level of the stream. The reason for choosing this model is that is the simplest model that I could think of that is still somewhat realistic and useful.

![Groundwater flow to a stream with uniform recharge](fig/1dim_flow_with_recharge.png)

### Boundary conditions (conceptual)

We impose two different boundary conditions:
- **Left boundary (stream):** fixed hydraulic head $h=h_0$.
- **Right boundary (watershed divide):** no-flow, so $\frac{\partial h}{\partial x}=0$.

In the numerical model this means:
- Set the first node equal to $h_0$.
- Set the watertable $h$ of the last node equal to the second-to-last node, which means that the gradient between these two nodes is zero.

### Analytical solution

For this problem, an analytical (exact) solution for the watertable exists:

$$h = h_{0} + \frac{R}{Kb} \left(Lx - \frac{1}{2} x^{2}\right)$$

where $R$ is recharge (m/s) and $L$ is the aquifer length (m).

We will use this analytical solution to verify our numerical model. See the (theory notebook)[groundwater_theory_derivations.ipynb] for the full derivation.

### Worked example: 5-node manual calculation

Before implementing the full model, let's work through a small example by hand to understand how the iterative solver works.

**Setup:** Consider a tiny aquifer with only 5 nodes (dx = 1000 m, L = 4000 m)

**Parameters:**
- K = 1×10⁻⁵ m/s
- b = 250 m
- R = 0.25 m/yr = 7.9×10⁻⁹ m/s
- h₀ = 250 m (left boundary)
- W = R/b = 3.16×10⁻¹¹ s⁻¹

**Step 1: Calculate the constant C**

The equation that we are trying to solve is:

$$h(x) = \frac{1}{2} \left( \frac{Wb \Delta x^2}{K b} + h(x+\Delta x) + h(x - \Delta x) \right)$$

We simplify this by defining a constant $C = \frac{W \Delta x^2}{K}$

Which means we can rewrite the equation to:

$$h(x) = \frac{1}{2} \left( C + h(x+\Delta x) + h(x - \Delta x) \right)$$

We first calculate C:

$$C = \frac{W \Delta x^2}{K} = \frac{3.16 \times 10^{-11} \times 1000^2}{1 \times 10^{-5}} = 3.16 \, \text{m}$$

**Step 2: Set initial guess for the watertable**

Start with h = 250 m everywhere (or zeros, it doesn't matter).

**Step 3: Apply the finite difference equation**

For each iteration:
- Node 0 (left boundary): h[0] = 250 m (fixed)
- Node 4 (right boundary): h[4] = h[3] (no-flow condition)
- Nodes 1, 2, 3 (interior): h[i] = 0.5 × (C + h[i-1] + h[i+1])

**Manual iteration example:**

| Iteration | h[0] | h[1] | h[2] | h[3] | h[4] |
|-----------|------|------|------|------|------|
| 0 (initial) | 250.00 | 250.00 | 250.00 | 250.00 | 250.00 |
| 1 | 250.00 | 251.58 | 251.58 | 251.58 | 251.58 |
| 2 | 250.00 | 252.37 | 253.16 | 253.16 | 253.16 |
| 3 | 250.00 | 252.76 | 254.34 | 254.74 | 254.34 |
| ... | ... | ... | ... | ... | ... |
| ∞ (converged) | 250.00 | 253.16 | 256.32 | 259.48 | 259.48 |

**Calculating h[1] at iteration 1:**
$$h[1] = 0.5 \times (3.16 + 250.00 + 250.00) = 0.5 \times 503.16 = 251.58 \, \text{m}$$

**Key insights:**
1. The solution converges gradually toward the steady-state, i.e. if you repeat the calculation long enough the numbers will reach a constant value
2. Boundary conditions remain fixed throughout
3. Interior nodes depend on their neighbors from the previous iteration
4. After many iterations (~1000), the solution stabilizes

This is exactly what the Python solver does, but for 51 nodes instead of 5!

## Part 2: Setting up the Numerical Model in Python

### Model Parameters

We will use the following parameters for our calculation. They represent the global median watershed, following an analysis of global data on recharge, watershed size and hydraulic conductivity by [https://doi.org/10.1038/ngeo2590](Gleeson et al., 2016)

| Parameter | Value | Unit |
| --------- | ----- | ---- |
| L (length) | 5000 | m |
| b (thickness) | 250 | m |
| h0 (initial head) | 250 | m |
| R (recharge) | 0.25 | m/yr |
| K (hydraulic conductivity) | 10⁻⁵ | m/s |
| Δx (grid spacing) | 100 | m |

We first import two modules that we need later on. Numpy for working with arrays, and matplotlib for making figures of our model results:

In [ ]:
# Import required modules
# Note: Lines starting with # are comments and are ignored by Python
# numpy for working with arrays:
import numpy as np
# matplotlib to make nice looking figures:
import matplotlib.pyplot as plt
# For examples of what you can make with matplotlib: https://matplotlib.org/

Next we define the parameters that we need later on:

In [ ]:
# Define model parameters
# Note: Use decimal points (5000.0 not 5000) to distinguish floats from integers
# All parameters should be in SI units

year = 365.25 * 24 * 60 * 60  # seconds in a year

L = 5000.0       # aquifer length (m)
b = 250.0        # aquifer thickness (m)
h0 = 250.0       # hydraulic head at the left boundary (m)
R = 0.25 / year  # recharge rate (m/s), converted from m/yr
K = 1e-5         # hydraulic conductivity (m/s)
dx = 100.0       # grid spacing (m)

print(f"Recharge rate: {R:.2e} m/s")

### Calculating the source term

The source term $W$ that we use in the numerical equation has units of $s^{-1}$. To convert recharge ($m s^{-1}$) to the source term we have to divide the total recharge that each grid cell receives, which is recharge multiplied by the width of each grid cell (dx), by the volume of each grid cell, which is the width (dx) times the height (b):

$$W = \frac{R \cdot \Delta x}{\Delta x \cdot b} = \frac{R}{b}$$

In [ ]:
# Calculate the source term W
W = R / b
print(f"Source term W: {W:.2e} s^-1")

### Setting up the numerical grid

For variables like distance (x) and the source term (W) we need to set up **arrays** (rows of numbers). This is similar to the columns you might use in a spreadsheet.

`np.arange(0, L + dx, dx)` generates an array starting at 0, ending at L, with spacing dx.

`np.ones_like(x)` creates a new array with the same length as x, filled with ones.

In [ ]:
# Set up the x-coordinates of the numerical grid
x = np.arange(0, L + dx, dx)

# Set up an array with the source term for each node
W_array = np.ones_like(x) * W

print(f"Number of nodes: {len(x)}")
print(f"x array: {x}")

### Python array indexing reference

**Important:** Python always starts counting at 0 (not 1).

Array indexing examples:
- `x[0]` - first element
- `x[-1]` - last element
- `x[-2]` - second-to-last element
- `x[1:-1]` - all elements except first and last
- `x[2:]` - all elements from index 2 onwards
- `x[:-2]` - all elements except the last two

These will be used in the solver function to implement the finite difference equation.

### Check your understanding (Part 2)
1. Why do we convert recharge from m/yr to m/s?
2. What happens to $W$ if aquifer thickness $b$ doubles?
3. Why does smaller $\Delta x$ usually improve accuracy but increase runtime?

### Hints for "Check your understanding" questions

<details>
<summary>Click to show hints (try answering first!)</summary>

**Part 2 Questions:**

1. **Why convert recharge?** All equations require consistent units. The groundwater flow equation uses SI units (m, s), so we must convert from m/yr to m/s.

2. **What happens if b doubles?** W = R/b, so if b doubles, W is halved. This means less source term per unit volume, resulting in a flatter watertable (smaller hydraulic gradient).

3. **Why does smaller Δx improve accuracy?** The finite difference approximation assumes the derivative is constant over the interval Δx. Smaller intervals better approximate the true continuous derivative. However, more nodes mean more calculations per iteration.

</details>

### Define the groundwater flow solver function

This function solves the steady-state groundwater flow equation iteratively using the finite difference method.

**About Python functions:**
A function is an isolated part of code that:
- Takes input variables (parameters)
- Does operations with those variables
- Returns results back to the main script

**About indentation:**
Python uses indents (empty spaces) to determine which code belongs to which function or loop. All indented code below `def solve_steady_state_gw_flow():` belongs to this function. You can add indents using the Tab key.

~~~python
# an example of indentation:

# this line is not indented
    # this line is indented
~~~

In [ ]:

def solve_steady_state_groundwater_flow(dx, K, W, h0, max_iterations=100000, max_error=1e-6):
    
    """
    this is a function where we will solve the steady-state diffusion equation
    for groundwater flow

    this function receives 4 variables from the main code: dx, K, W, h0
    plus an optional variable n_iterations. The default value for this
    variable if not specified otherwise is 1000

    """
    
    # check the number of nodes in our numerical model:
    n_nodes = len(W)
    
    # set u an array to store the variable (ie, hydraulic head or temperature)
    h_new = np.zeros(n_nodes)

    # and set up a similar array to store the variable value of the previous 
    # iteration step
    h_old = h_new.copy()

    # calculate the right hand term of the finite difference equation:
    C = W * dx**2 / K
    
    max_error_observed = 1.0e10
    iteration = 0
    
    # set up a loop to repeat the calculation untill the solution does not change anymore
    while (max_error_observed > max_error) and (iteration < max_iterations):
        
        # set up a new for loop to go through all the grid cells:
        for i in range(n_nodes):
            
            # check if we are at the left-hand boundary of the model domain
            if i == 0:
                # complete the next line and remove the comment sign (#)
                #h_new[0] = ....

            # check if we are at the right hand boundary instead
            elif i == n_nodes - 1:
                # complete the next line and remove the comment sign (#)
                #h_new[-1] = .... a function of h_old[-2]

            else:
                # add the equation for the middle nodes here:
                #h_new[i] = ..... a function of C[i], h_old[i-1] and h_old[i+1]
    
        # find the maximum difference between old and new h values to check for convergence
        max_error_observed = np.max(np.abs(h_new - h_old))

        iterations += 1

        # copy the new u into the u array for the previous timestep:
        h_old = h_new.copy()
    
    print(f"Converged in {iteration} iterations with max error {max_error_observed:.2e}")
    
    # done, you can now pass on the calculated value of u back to the main
    # part of the code:
    return h_new

## Using the groundwater flow function:

Next we can already call our function (`solve_steady_state_groundwater_flow`) that solves the steady-state groundwater equation. 

However, the function is not complete yet. If you go through the function you will notice a few lines that still have to be completed. The function starts with creating two new arrays, ``h_new`` and ``h_old``. These store the value of the variable you are trying to solve, which in this case is hydraulic head. The function solves the equation iteratively. After each iteration time step the newly calculated value ``h_new`` is copied to ``h_old`` and the iteration is repeated. 

The iterations are executed using a so called for loop. The following line:

~~~~python
for n_iter in range(n_iterations):
~~~~

means that any code that is below this line and that is indented is repeated ``n_iterations`` times.

There is a second for loop that is inside the first for loop, which makes sure we go over each node in our model: 

~~~~python
    # set up a for loop to repeat the calculation n_iterations times
    for n_iter in range(n_iterations):
        
        # set up a new for loop to go through all the grid cells:
        for i in range(n_nodes):
~~~~

The code in this second for loop does the actual calculation of the hydraulic head for each node and each iteration.

## Assignment 1: Complete the equations

Next we have to make sure that the groundwater flow equation is solved correctly. **Complete the following three lines in the solve_steady_state_groundwater_flow function:**

1. one line where the hydraulic head at the left hand side of the model domain is calculated
2. one line that calculates value on the right hand side of the model domain
3. and one line where you calculate hydraulic head in the remaining nodes in the middle. 


Now go ahead and first try to complete the line starting with ``#h_new[0] = ``:

~~~~python
# check if we are at the left-hand boundary of the model domain
if i == 0:
    # complete the next line and remove the comment sign (#)
    #h_new[0] = ....
~~~~

the variable ``h_new[0]`` means the value of ``h_new`` at the first node, which has node number 0. Note that Python always starts to count at 0, and not 1 like for instance in Matlab. We assigned a specified hydraulic head to the first node. The specified head is passed to the function as the variable ``h0``.

Next, try to complete the line for the nodes in the middle:

~~~~python
else:
    # add the equation for the middle nodes here:
    #h_new[i] = ..... a function of C[i], h_old[i-1] and h_old[i+1]
~~~~

Look up the correct equation in the introduction section of this notebook, modify the line and remove the # sign before the line to make it active. `h_new[i]` means the value of h_new at node number `i`. Note that i is part of the for loop: ``for i in range(n_nodes):``. This means that everything below this loop is repeated and `i` is increased with one after completing each loop. ``h_old[i-1]`` means the value of h_old at node number `i-1`, which is the node before node `i`. And similarly ``h_old[i+1]`` means the value of h_old at node `i+1`, the next node.

Next we have to make sure that the right hand boundary acts as a no flow boundary. We do this by making sure that the hydraulic head at the last grid cell always had the same value as the second last grid cell, i.e., making sure that the value in the last node (``h_new[-1]``) is always the same as the second last node (``h_new[-2]``):

~~~~python
    # check if we are at the right hand boundary instead
    elif i == n_nodes - 1:
        # complete the next line and remove the comment sign (#)
        #h_new[-1] = .... a function of h_old[-2]
~~~~

The index ``[-1]`` is shorthand for the last item in an array. So ``h_new[-1]`` is the value of hydraulic head for the right most grid cell, and ``h_new[-2]`` is the second last node, etc...

## Running the model

Make sure you complete and run the function above and that the jupyter notebook does not generate an error if you do so. The reason is usually a typo or a wrong indentation. Try to fix this or call for help from your instructor if you cannot figure out the error.

Now try to run the model code by running the cell below. Watch the values of h increase towards a steady-state value (hopefully). Increase the number of iterations in the function if you need more steps to reach steady-state.

In [ ]:
# call the steady-state groundwater flow function to calculate h
h = solve_steady_state_groundwater_flow(dx, K, W_array, h0)

If everything works ok: congrats! You just wrote your very first numerical model code. You can inspect the modelled values of hydraulic head by making a new code cell by slecting the plus button above, and typing ``print(h)`` in this cell. If you want to only see part of the h array (which contains the modeled hydraulic head), you can for instance type ``print(h[10:20])`` to see the values of h for node 10 to 20.

If the code does not work: Do not panic. Go over your code to make sure there are no typos etc, you did not forget to define any variables, indentations are ok, etc... If the code is still not behaving: try to get the attention of your instructor or shout help.

## Testing boundary conditions

Before continuing to run the model, let's verify that our boundary conditions work correctly with a simple test.

In [ ]:
# Test boundary conditions with a simple 5-node case
print("Boundary condition validation test:")
print("-" * 50)

# Create a small test case
x_test = np.array([0., 1000., 2000., 3000., 4000.])
W_test = np.ones_like(x_test) * W
h_test = solve_steady_state_groundwater_flow(1000., K, W_test, h0, n_iterations=5000)

print(f"Left boundary (fixed head):")
print(f"  Expected: h[0] = {h0} m")
print(f"  Actual:   h[0] = {h_test[0]:.6f} m")
print(f"  ✓ Boundary condition satisfied: {abs(h_test[0] - h0) < 1e-10}")

print(f"\nRight boundary (no-flow, dh/dx = 0):")
print(f"  Expected: h[n-1] ≈ h[n-2]")
print(f"  h[{len(h_test)-1}] = {h_test[-1]:.6f} m")
print(f"  h[{len(h_test)-2}] = {h_test[-2]:.6f} m")
print(f"  Difference: {abs(h_test[-1] - h_test[-2]):.2e} m")
print(f"  ✓ Boundary condition satisfied: {abs(h_test[-1] - h_test[-2]) < 0.01}")

print("\n" + "=" * 50)
print("Both boundary conditions working correctly!")
print("=" * 50)

## Adding the analytical solution

One good habit when running numerical models is to try and always find an analytical solution to test whether your numerical model is behaving well. Try to implement the analytical solution for the groundwater table that was shown in exercise 1. 

## Assignment 2: 
**Complete the line below that starts with ``#h_an = `` by adding the analytical solution equation discussed in the introduction, and uncomment this line (remove the # at the start of the line)**

In [ ]:
# analytical solution for steady-state groundwater flow
# complete the line below by writing the equation for h in correct Python syntax
# note that multiplication is done with *, exponentiaton by **, adding by +

#h_an = h0 + ...

## Graphical output

A computer model is really not complete without colourful pictures of the model result. Making nice-looking figures with Python is easy thanks to the matplotlib module that we have imported already. For an overview of what you can do with matplotlib surf to the website and look at the gallery: <http://matplotlib.org/gallery.html>.

There are already a number of lines of code at the bottom of the script that will generate a figure of the model results. 

The following line creates a new figure with one panel:


In [ ]:
def plot_results(x, h_an, h):
    
    """
    this is a function to plot the results of the groundwater model

    it receives three variables from the main code: x, h_an, h

    """

    # set up a figure with one panel
    fig, panel = plt.subplots(1, 1)

    # plot the analytical solution
    panel.plot(x, h_an, color='green', label='h, analytical')

    # and the numerical solution, add the right variables and uncomment (remove #) the next line:
    #panel.plot(....)

    # make the figure nicer:
    panel.set_xlabel('Distance (m)')
    panel.set_ylabel('Elevation (m)')
    panel.legend(loc='upper left', fontsize='medium', frameon=False)

    # save the figure:
    fig.savefig('simulated_h.png')

    # and show it
    plt.show()

    return

In [ ]:
plot_results(x, h_an, h)

note that you can copy and paste this command elsewhere to make additional figures in this notebook for different model runs

## Assignment 3 

**Complete the code above to make a figure that contains the analytical and numerical values of h.**

## Making the code faster
For loops in general make your code relatively slow. We can use numpy's functionality to avoid the inner for loop that cycles over the grid cells and try to calculate the new value of *h* for all nodes in one go at each timestep. We will implement the faster code in a new function called ``solve_steady_state_groundwater_flow_faster`` which you can find below.

For this we need to remove the inner for loop (remove the line ``for i in range(n_nodes):``, and unindent the lines in this for loop) and replace the equations for *h*. The two lines for the boundary conditions can remain the same, since they do not depend on the value of ``i``, which tracks the node number. For all the nodes in between the boundary conditions, we can calculate *h* like this:

~~~~python
h_new[1:-1] = ... insert an equation here with the variables C[1:-1], h_old[2:] and h_old[:-2]
~~~~

In this piece of code ``h_new[1:-1]`` means all grid cells except the first and last ones. ``h_old[2:]`` means all grid cells, except the first two, and ``h_old[:-2]`` means all grid cells except the last two. This statement does exactly the same as our for loop earlier, but many times faster.



In [ ]:
def solve_steady_state_groundwater_flow_faster(dx, K, W, h0, max_iterations=100000, max_error=1e-6):
    
    """
    this is a function where we will solve the steady-state diffusion equation
    for groundwater or heat flow

    this function receives 4 variables from the main code: dx, K, W, u0
    plus an optional variable n_iterations. The default value for this
    variable if not specified otherwise is 1000

    """
    
    # check the number of nodes in our numerical model:
    n_nodes = len(W)
    
    # set u an array to store the variable (ie, hydraulic head or temperature)
    h_new = np.zeros(n_nodes)

    # and set up a similar array to store the variable value of the previous 
    # iteration step
    h_old = h_new.copy()

    # calculate the right hand term of the finite difference equation:
    C = W * dx**2 / K
    
    max_error_observed = 1.0e10
    iteration = 0
    
    # set up a for loop to repeat the calculation n_iterations times
    while (max_error_observed > max_error) and (iteration < max_iterations):
        
        # complete the next line and remove the comment sign (#)
        #h_new[0] = ....

        # complete the next line and remove the comment sign (#)
        #h_new[-1] = .... a function of h_old[-2]

        # add the equation for the middle nodes here:
        #h_new[1:-1] = ..... a function of C[1:-1], h_old[2:], h_old[:-2]

        # find the maximum difference between old and new h values to check for convergence
        max_error_observed = np.max(np.abs(h_new - h_old))

        # copy the new u into the u array for the previous timestep:
        h_old = h_new.copy()

        iteration += 1
    
    print(f"Converged in {iteration} iterations with max error {max_error_observed:.2e}")
    
    # done, you can now pass on the calculated value of u back to the main
    # part of the code:
    return h_new

In [ ]:
h = solve_steady_state_groundwater_flow_faster(dx, K, W_array, h0, n_iterations=1000)

max_difference = np.max(np.abs(h - h_an))

print(f"Calculated h (faster function): {h}")
print(f"difference between numerical and analytical solution: {np.max(np.abs(h - solve_steady_state_groundwater_flow(dx, K, W_array, h0)))}")


## Assignment 4 

**4.1 Complete the lines to implement the new faster code above and run the numerical model again**

**4.2 Try to run your model with two or more more values for grid size ($\Delta x$) and report the model error for each model run.** 

**4.3 Discuss which grid size would be sufficient for an accurate but also fast model of groundwater flow in this case?**

**4.4: What is the water flux $Q$ to the stream? And what is this flux halfway the model domain?**

**4.5 What is the shape of the watertable? Why is the watertable not linear? Hints: try to think about how the flux (*Q*) changes over the model domain? Is it constant? And what is the relation between flux and hydraulic gradient in Darcy's law?**

### Simulate the effect of pumping

One major advantage of numerical solutions over analytical solutions is that parameters such as hydraulic conductivity or recharge do not have to be constant but can be varied over the model domain. We will use this flexibility to adjuist the source term ($W$) to simulate the effect of pumping on water levels.

## Assignment 5

**5.1 Add a pump at 1000 m distance to the stream, and simulate a well discharge that is exactly equal to the total amount of recharge that the aquifer receives (which is the recharge times the total length of the watershed)**

Note that the pumping rate in this case has the unit $m^2 s^{-1}$ and not $m^3 s^{-1}$ because we are working in two dimensions, instead of three.

You can add a pumping well by modifying source term ($W$). For the grid cell that contains the pumping well (at x=1000 m) change the source term $W$ to a value that is equal to recharge minus pumping at the location of the well. Make sure to convert the units of pumping from $m^2 s^{-1}$ to $s^{-1}$ by dividing the number by the area of one grid cell ($\delta x * b$).

**5.2 Report the hydraulic head at the pumping node and the hydraulic head at the stream**

**5.3 Describe what the long-term consequence is of pumping at this rate for the hydrology of this watershed**

In [ ]:
# calculate the location of the well in the grid
#well_index = int(...)

# adjust the value of W in the grid
W_pumping = W_array.copy()
#W_pumping[well_index] = ...

# copy-paste the code here to run the model again with the well included, and use W_pumping instead of W
# ....

# and copy-paste the plotting function call to make a new figure showing the effect of the well
# ...

## Model calibration

In many cases the exact value of model parameters such as hydraulic conductivity or recharge in a particular aquifer or geologic unit are unknown. One way to get around this is to first guess these parameters and then keep on adjusting the value of these parameters until the simulated hydraulic head are close to those observed in boreholes the field. This is called model calibration or inverse modeling.

One of the advantages of Python is that over the last two decades or so a lot of people have written a huge variety of Python modules that add all kinds of different useful functionality. One such module is called Scipy, <http://www.scipy.org>. This is a large collection of mathematical and scientific functions. Scipy contains a set of functions that deal with model calibration. We will use this to calibrate hydraulic conductivity in our numerical model. 

The automated calibration module needs a second function that compares the modelled value of ``h`` to an observed value in a borehole and then returns the model error, i.e. the difference between the observed and modelled value. This function looks like this:

In [ ]:
def model_error(params, dx, W, h0, x_obs, h_obs):
    
    K = params[0]

    # calculate the location index of the observation point in the grid
    ix_obs = int(x_obs / dx)
    
    # calculate the model predicted value of h
    h_pred = solve_steady_state_groundwater_flow_faster(dx, K, W, h0)
    
    # calculate the absolute error between model and observed h
    h_error = np.abs(h_pred[ix_obs] - h_obs)
    
    print('h error = ', h_error)

    return h_error

As you can see this function runs the model first and then calculates the absolute difference between *h* at some distance `x_obs` and some observed value ``h_obs``. The variable ``params`` is a list that contains all the parameters that we want to calibrate automatically. In this case we only calibrate ``K``, so ``params`` can be a list with only one value. We use the absolute model error, since the calibration function that we use is a functions that tries to minimise a value, ie.: it runs the function ``model_error`` again and again to find the lowest value of ``h_error``.

Now all we need to run automatic calibration is to call one of scipy's automatic calibration functions in the main code.




In [ ]:
import scipy.optimize as opt

# the distance of the observation point in meters
#x_obs = ...

# the observed value of h at that location
#h_obs = ...

params = [K]
params_calibrated = opt.fmin(model_error, params, args=(dx, W_array, h0, x_obs, h_obs))
K_calibrated = params_calibrated[0]

print('new calibrated value of K = ', K_calibrated)

now we rerun the numerical model with the updated value of K and make a figure with the results:

In [ ]:
# rerun the model with the calibrated K
h = solve_steady_state_groundwater_flow_faster(dx, K_calibrated, W_array, h0)

# copy the code that was used to make the previous figure here to make a new figure with the calibrated K:
# ....


## Assignment 6:


Now assume that we have installed a borehole exactly at the watershed boundary at 5000 m distance of the stream. We measured the hydraulic head, which is found to be exactly 100.0 m above the level of the stream. Use this new information to estimate a new value of hydraulic conductivity, by calibrating your numerical groundwater model. Also make sure that you do not use the pumping well that you have added for question 2.

**6.1 Run the model with the new automatic calibration function and report the calibrated value of K. Use the new value of ``K`` to model hydraulic head and make a figure of this.**

**6.2 Given this value, what type of material could the geologic unit consist of?** Look up typical values for permeability (k) in Gleeson et al. (2011)(https://doi.org/10.1029/2010GL045565)). Note that permeability (usually denoted by k) is approximately $10^7$ times smaller than hydraulic conductivity (K), or $k = \dfrac{K}{10^7}$


## Last check

Before handing in the assignment, make sure you have checked the following points:

- Restart the notebook and run all cells
- Make sure your code runs without errors and all the results look good.
- Make sure you have answered all assignments
- Rename the notebook to `geov212_exercise1_lastname_firstname.ipynb` where you replace lastname and firstname with your own last and first names. 
- Before and after handing in the assignment on mitt.uib: double check that you have handed in the correct file and all looks good

## Summary

Congrats! You wrote your first computer model!

In this exercise we:

1. Learned about the finite difference method for solving the groundwater flow equation
2. Implemented a numerical groundwater model in Python
3. Validated the model against an analytical solution
4. Performed model calibration using observed data
5. Analyzed the effect of measurement errors on model calibration